# Cleaning the PSGC Dataset

In [1]:
import pandas as pd
import re

In [2]:
psgc = pd.read_csv('psgc.csv.gz', dtype={'Code': str})

In [3]:
psgc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43796 entries, 0 to 43795
Data columns (total 8 columns):
Code                                 43796 non-null object
Name                                 43796 non-null object
Inter-Level                          43794 non-null object
City Class                           145 non-null object
Income
Classification                1715 non-null object
Urban / Rural (based on 2010 CPH)    42046 non-null object
POPULATION
(2015 POPCEN)             43794 non-null object
Unnamed: 7                           7 non-null object
dtypes: object(8)
memory usage: 2.7+ MB


Drop unused columns:

In [6]:
#psgc = psgc.drop(columns=['City Class', 'Income\nClassification', 'Urban / Rural (based on 2010 CPH)', 'POPULATION\n(2015 POPCEN)', 'Unnamed: 7'])
psgc = psgc.loc[:,['Code','Name','Inter-Level']]

In [7]:
psgc['code'] = psgc.pop('Code')
psgc['location'] = psgc.pop('Name').str.strip()
psgc['interlevel'] = psgc.pop('Inter-Level').str.strip()

In [8]:
psgc['interlevel'].value_counts()

Bgy       42044
Mun        1489
City        145
Prov         81
Reg          17
SubMun       14
Dist          4
Name: interlevel, dtype: int64

In [9]:
psgc

,code,location,interlevel
0,010000000,REGION I (ILOCOS REGION),Reg
1,012800000,ILOCOS NORTE,Prov
2,012801000,ADAMS,Mun
3,012801001,Adams (Pob.),Bgy
4,012802000,BACARRA,Mun
5,012802001,Bani,Bgy
6,012802002,Buyon,Bgy
7,012802003,Cabaruan,Bgy
8,012802004,Cabulalaan,Bgy
9,012802005,Cabusligan,Bgy


## Capitalize the location field

In [10]:
psgc['location'] = psgc['location'].str.upper()

## Create a duplicate of the original PSGC dataframe

In [11]:
og_psgc = psgc.copy()

## Helpers

In [12]:
pat_expand_location = re.compile('(.+)\(+([^\(\)]+)\)*')  # ¯\_(ツ)_/¯ 
def expand_location(row):
    '''
    Extracts location names inside parens and expands it
    to the new column `alt_location`
    '''
    m = pat_expand_location.search(row['location'])
    if m:
        location, alt_location = m.groups()
        return {
            **row,
            'location': location.strip(),
            'alt_location': alt_location.strip(),
        }
    return row

SyntaxError: invalid syntax (<ipython-input-12-8dd2f34fb093>, line 11)

## Clean regions

In [ ]:
regions = psgc[psgc['interlevel'] == 'Reg'].copy()

In [ ]:
regions

Alternate names inside parens so we expand those out to a new column named `alt_location`.

In [ ]:
regions = regions.apply(expand_location, axis=1, result_type='expand').fillna('')

In [ ]:
regions

## Clean provinces

In [ ]:
provinces = psgc[psgc['interlevel'] == 'Prov'].copy()

In [ ]:
provinces

Seems normal... But let's check for parens just in case:

In [ ]:
provinces[provinces['location'].str.contains('[\(\)]')]

Sneaky alternate names!

In [ ]:
provinces = provinces.apply(expand_location, axis=1, result_type='expand').fillna('')

In [ ]:
provinces

## Clean districts

In [ ]:
districts = psgc[psgc['interlevel'] == 'Dist'].copy()

In [ ]:
districts

No one writes `NTH DISTRICT (Not a Province)` in their addresses...

In [ ]:
districts['location'] = (districts['location']
                         .str.replace(',.+DISTRICT \(Not a Province\)', '')
                         .str.strip())

In [ ]:
districts

## Clean municipalities

In [ ]:
municipalities = psgc[psgc['interlevel'] == 'Mun'].copy()

In [ ]:
municipalities

A few alternate names but what are those `(Capital)` ones?

In [ ]:
municipalities[municipalities['location'].str.contains('[\(\)]')]

Apparently, they are capitals of their provinces. Safe to strip!

In [ ]:
municipalities['location'] = municipalities['location'].str.replace('\(Capital\)', '').str.strip()

In [ ]:
municipalities = municipalities.apply(expand_location, axis=1, result_type='expand').fillna('')

In [ ]:
municipalities

## Clean cities

In [ ]:
cities = psgc[psgc['interlevel'] == 'City'].copy()

In [ ]:
cities

Here we go with the `(Capital)` thing again.

In [ ]:
cities['location'] =  cities['location'].str.replace('\(Capital\)', '').str.strip()

Checking if there are still stuff with parens:

In [ ]:
cities[cities['location'].str.contains('[\(\)]')].head()

A few alterate names!

In [ ]:
cities = cities.apply(expand_location, axis=1, result_type='expand').fillna('')

Now what about those `CITY` pre/suffixes?

In [ ]:
cities[cities['location'].str.contains('CITY')]

In [ ]:
cities['location'] = (cities['location']
 .str.replace('^.*CITY OF', '')
 .str.strip()
 .str.replace('CITY$', '')
 .str.strip())

In [ ]:
cities

## Clean sub-municipalities

In [ ]:
sub_municipalities = psgc[psgc['interlevel'] == 'SubMun'].copy()

In [ ]:
sub_municipalities

Nothing special!

## Clean barangays

In [ ]:
barangays = psgc[psgc['interlevel'] == 'Bgy'].copy()

In [ ]:
barangays

We see alternate names again but notice the `(Pob.)` suffixes. A quick Google search shows that it's short for `Poblacion` which is used to denote the commercial and industrial center of a city.

In [ ]:
barangays['location'] = (barangays['location']
                         .str.replace('\(?POB\.\)?', '')
                         .str.strip())

Let's check for more weird characters:

In [ ]:
barangays[barangays['location'].str.contains('[^A-ZÑ0-9\-.\/\(\) ]')]

Aside for alternate names, there are those starting with `BGY. NO. X,`:

In [ ]:
barangays[barangays['location'].str.contains('^B[GR]Y. NO.')]

Let's set the `BGY. NO. X` bit as `alt_location`:

In [ ]:
pat_expand_bgy_location = re.compile('(B[GR]Y. NO. \d+\-?\w?),? (.+)')
def expand_bgy_location(row):
    m = pat_expand_bgy_location.search(row['location'])
    if m:
        alt_location, location = m.groups()
        return {
            **row,
            'location': location.strip(),
            'alt_location': alt_location.strip(),
        }
    return expand_location(row)
barangays = barangays.apply(expand_bgy_location, axis=1, result_type='expand').fillna('')

Check if we got all (there should be 80):

In [ ]:
barangays[barangays['alt_location'].str.contains('B[GR]Y. NO.')]

Another check for weird stuff:

In [ ]:
barangays[barangays['location'].str.contains('[^A-ZÑ0-9 \-.,\/]')]

Trim weird stuff:

In [ ]:
barangays['location'] = barangays['location'].str.strip('[\*\(\)]')
barangays['alt_location'] = barangays['alt_location'].str.strip('[\*\(\)]')

Last check!

In [ ]:
barangays[barangays['location'].str.contains('[^A-ZÑ0-9 \-.,\/]')]

## ARMM: Cotabato and Isabela City

In [ ]:
armm = psgc[psgc['interlevel'].isnull()].copy()
armm

In [ ]:
armm['location'] = armm['location'].str.replace('\(Not a Province\)', '')
armm

In [ ]:
armm['location'] = (armm['location']
 .str.replace('^.*CITY OF', '')
 .str.strip()
 .str.replace('CITY$', '')
 .str.strip())
armm

## All together now

In [ ]:
merged = pd.concat([
    regions,
    provinces,
    districts,
    municipalities,
    cities,
    sub_municipalities,
    barangays,
    armm
], sort=True).sort_index().fillna('')

Are counts still correct?

In [ ]:
psgc['interlevel'].value_counts()

In [ ]:
merged['interlevel'].value_counts()

In [ ]:
display(len(merged), len(psgc))

Extract alternate locations into a new dataframe:

In [ ]:
alt = merged[merged['alt_location'] != ''].copy()
alt['location'] = alt.pop('alt_location')

In [ ]:
alt

Concat alternate locations and do final cleanup on location name:

In [ ]:
clean_psgc = (pd.concat([merged.drop(columns=['alt_location']), alt], ignore_index=True)
              .sort_values('code')
              .reset_index(drop=True))

Normalize `Ñ` and remove remaining `*`s:

In [ ]:
clean_psgc['location'] = (clean_psgc['location']
                          .str.replace('Ñ', 'N')
                          .str.replace('\*', ''))

Normalize numbers:

In [ ]:
spanish = clean_psgc[clean_psgc['location'].str.contains(' (UNO|DOS|TRES|KUATRO|SINGKO)$')].copy()

In [ ]:
for i, s in enumerate([
    'UNO',
    'DOS',
    'TRES',
    'KUATRO',
    'SINGKO',
]):
    spanish['location'] = spanish['location'].str.replace(' {}$'.format(s), ' {}'.format(i + 1))
spanish

In [ ]:
roman = clean_psgc[clean_psgc['location'].str.contains('\s(X{0,3})(IX|IV|V?I{0,3})$')].copy()

In [ ]:
for i, s in enumerate('I,II,III,IV,V,VI,VII,VIII,IX,X,XI,XII,XIII,XIV,XV,XVI,XVII,XVIII,XIX,XX,XXI,XXII'.split(',')):
    roman['location'] = roman['location'].str.replace(' {}$'.format(s), ' {}'.format(i + 1))
roman

Provide alternate names for locations with President names

In [ ]:
president = clean_psgc[clean_psgc.location.str.contains('PRES\.', flags=re.IGNORECASE)].copy()
president['location'] = president['location'].str.replace('^PRES\.', 'PRESIDENT')

# Add alternative names to the Manila districts

In [ ]:
alt_ncr_as_manila = clean_psgc[clean_psgc.interlevel == 'Dist'].copy()
alt_ncr_as_manila['location'] = alt_ncr_as_manila['location'].str.replace('NCR', 'MANILA')

alt_ncr_as_metro_manila = clean_psgc[clean_psgc.interlevel == 'Dist'].copy()
alt_ncr_as_metro_manila['location'] = alt_ncr_as_metro_manila['location'].str.replace('NCR', 'METRO MANILA')

alt_ncr_abbvr_expanded = clean_psgc[clean_psgc.interlevel == 'Dist'].copy()
alt_ncr_abbvr_expanded['location'] = alt_ncr_abbvr_expanded['location'].str.replace('NCR', 'NATIONAL CAPITAL REGION')

alt_ncr_as_metropolitan = clean_psgc[clean_psgc.interlevel == 'Dist'].copy()
alt_ncr_as_metropolitan['location'] = alt_ncr_as_metropolitan['location'].str.replace('NCR', 'METROPOLITAN MANILA')

alt_districts = pd.concat([alt_ncr_as_manila,
                           alt_ncr_abbvr_expanded,
                           alt_ncr_as_metropolitan,
                           alt_ncr_as_metro_manila], ignore_index=True)
alt_districts

# Add Ñ -> N as an alternate name

In [ ]:
enye = clean_psgc[clean_psgc.location.str.contains('Ñ')].copy()
enye.head()

In [ ]:
enye['location'] = enye['location'].str.replace('Ñ', 'N')
enye.head()

# Concat the alternates to the main dataframe

In [ ]:
clean_psgc = (pd.concat([clean_psgc, spanish, roman, president, alt_districts, enye], ignore_index=True)
              .sort_values('code')
              .reset_index(drop=True))

Last check for weird stuff!

In [ ]:
clean_psgc[clean_psgc['location'].str.contains('[^A-Z0-9 \-.,\']')]

We can probably still split with `&` and `/` but this is good enough for now.

## Combine the cleaned up PSGC and remove the duplicates

In [ ]:
og_psgc['original'] = True
clean_psgc['original'] = False

clean_psgc = pd.concat([og_psgc, clean_psgc], sort=False)
clean_psgc.drop_duplicates(subset=['code', 'location', 'interlevel'], inplace=True)
clean_psgc.sort_values('code', inplace=True)

Check that we have both the original name and the alternate ones

In [ ]:
clean_psgc[clean_psgc.code.str.contains('086000000')]

In [ ]:
clean_psgc.to_csv('clean-psgc.csv.gz', index=False, compression='gzip')

And we're done!